# Introduction

In [1]:
import ipywidgets as widgets
from soniscope_jupyter import LensWidget, ALLOWED_SHAPES
import pandas as pd

## simple example

In [2]:
plot = LensWidget(substrate_width=200, substrate_height=200)
plot

LensWidget(substrate_height=200, substrate_width=200)

In [3]:
df = pd.DataFrame({'var1': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8},
                   'var2': {0: 1, 1: 4, 2: 9, 3: 16, 4: 25, 5: 36, 6: 49, 7: 64}})
plot.set_data(df, "var1", "var2")

In [4]:
plot.x_field = "var2"

## Interactive Demo

We are using a dataset Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg, <https://doi.org/10.1007/s13748-013-0040-3>.

See <https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset>

In [5]:
# daily_file = r'c:\Users\arind\Documents\scm\sonivis-jupyter\BikeData\day.csv'
daily_file = 'bike_day_semantic.csv'
daily = pd.read_table(daily_file, sep=',')  # , low_memory=False
daily.head(3)

,instant,dteday,season,mnth,holiday,weekday,workingday,weathersit,windspeed,casual,registered,cnt,temp °C,atemp °C,hum %,year
0,1,2011-01-01,winter,1,0,Saturday,0,"Mist + Cloudy, Mist + Broken clouds, Mist + Fe...",10.749882,331,654,985,8.175849,7.999250,80.5833,2011
1,2,2011-01-02,winter,1,0,Sunday,0,"Mist + Cloudy, Mist + Broken clouds, Mist + Fe...",16.652113,131,670,801,9.083466,7.346774,69.6087,2011
2,3,2011-01-03,winter,1,0,Monday,1,"Clear, Few clouds, Partly cloudy, Partly cloudy",16.636703,120,1229,1349,1.229108,-3.499270,43.7273,2011


In [6]:
# get possible colums for axes
numeric_cols = daily.select_dtypes(include='number').columns.values.tolist()
categoric_cols = ['', 'weathersit', 'workingday', 'weekday', 'mnth', 'season', 'yr']

In [7]:
plotBike = LensWidget(daily, 'temp °C', 'hum %')
sliderSize = widgets.FloatSlider(description='Size', min=0.01, max=1.5, step=0.01, readout_format='.0%')
radioShape = widgets.RadioButtons(options=ALLOWED_SHAPES,
    layout={'width': 'max-content'}, # If the items' names are long
    description='Shape:'
)
drpX = widgets.Dropdown(options=numeric_cols, value='temp °C', description='X-Axis:',)
drpY = widgets.Dropdown(options=numeric_cols, value='hum %', description='Y-Axis:',)
drpC = widgets.Dropdown(options=categoric_cols, value='', description='Color:',)

outputBike = widgets.Output()
l = widgets.link((plotBike, 'size'), (sliderSize, 'value'))
l2 = widgets.link((plotBike, 'shape'), (radioShape, 'value'))
l3 = widgets.link((plotBike, 'x_field'), (drpX, 'value'))
l4 = widgets.link((plotBike, 'y_field'), (drpY, 'value'))
l5 = widgets.link((plotBike, 'color_field'), (drpC, 'value'))

display(plotBike, 
        widgets.HBox([
            widgets.VBox([drpX, drpY,drpC,sliderSize]), 
            radioShape]), 
        outputBike)

# def on_plot_clicked(widget, payload, _b):
#     with outputBike:
#         # print("Plot clicked. ")
#         print(widget, payload)

# plotBike.on_msg(on_plot_clicked)

def on_lens_clicked(widget, x: float, y: float, filtered: pd.DataFrame, distances: pd.Series):
    with outputBike:
        # print("Plot clicked. ")
        # print(widget)
        print("at", x, y, "found", filtered.shape[0], "tuple")
        item = filtered.index[0]
        print("1st tuple", filtered.at[item, 'temp'], filtered.at[item, 'hum'], "has distance", distances.at[item]) 

plotBike.on_lens_click(on_lens_clicked)

LensWidget(x_field='temp °C', y_field='hum %')

Output()

In [8]:
plotBike.substrate_width = 300
plotBike.substrate_height = 200

In [9]:
plotBike.x_field = "cnt"
plotBike.shape="square"